In [1]:
import numpy as np
import pandas as pd
from river import optim
from river import metrics
from river.drift.binary import EDDM
from sail.models.auto_ml.tune import SAILTuneGridSearchCV
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.models.river.linear_model import LinearRegression
from sail.models.auto_ml.auto_pipeline import SAILAutoPipeline
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sail.drift_detection.drift_detector import SAILDriftDetector
from sail.transformers.river.preprocessing import StandardScaler

#### Load Data


In [2]:
X = pd.read_csv("../../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
linear_reg = LinearRegression(optimizer=optim.SGD(0.1))
random_forest = AdaptiveRandomForestRegressor()

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", "passthrough"),
]
sail_pipeline = SAILPipeline(steps=steps, scoring=metrics.R2)

#### HPO Params


In [5]:
params_grid = [
    {
        "regressor": [linear_reg],
        "regressor__l2": [0.1, 0.9],
        "regressor__intercept_init": [0.2, 0.5],
    },
    {
        "regressor": [random_forest],
        "regressor__n_models": [10, 15, 20]
    },
]

#### Create Model Instance


In [6]:
auto_pipeline = SAILAutoPipeline(
    pipeline=sail_pipeline,
    pipeline_params_grid=params_grid,
    search_method=SAILTuneGridSearchCV,
    search_method_params={
        "num_cpus_per_trial": 1,
        "max_iters": 1,
        "early_stopping": False,
        "mode": "max",
        "scoring": "r2",
        "pipeline_auto_early_stop": False,
        "keep_best_configurations": 2
    },
    search_data_size=1000,
    pipeline_strategy="PrequentialTraining",
)

[2023-07-11 21:13:16:730] - INFO - SAIL : Pipeline Strategy [PrequentialTraining] created with actions: ['DATA_COLLECTION', 'FIND_BEST_PIPELINE', 'PARTIAL_FIT_PIPELINE']


#### Start Training


In [7]:
y_preds = []
y_true = []
batch_size = 50

for start in range(0, 1000, batch_size):

    end = start + batch_size

    X_train = X.iloc[start:end]
    y_train = y.iloc[start:end]

    if end > 1000: # search_data_size is 1000
        preds = auto_pipeline.predict(X_train)
        if preds is not None:
            y_preds.extend(list(preds))
            y_true.extend(list(y_train))

    auto_pipeline.train(X_train, y_train)


2023-07-11 21:13:38,583	INFO tune.py:945 -- Total run time: 17.53 seconds (17.52 seconds for the tuning loop).
SAIL Pipeline Score: 100%========== [Points: 1000/1000, Elapsed:0.0058s, Metric=R2, Batch Size=1000, Score=-1.35]  
SAIL Pipeline Score: 100%========== [Points: 1000/1000, Elapsed:0.0102s, Metric=R2, Batch Size=1000, Score=0.843]
SAIL Pipeline fit: 100%========== [Steps: 3/3, ETA: 00:03<00:00, Elapsed:3.064s, Batch Size=1000, Score=0.843]            
[2023-07-11 21:13:43:960] - INFO - SAIL : Pipeline tuning completed. Disconnecting Ray cluster...
[2023-07-11 21:13:43:960] - INFO - SAIL : Found best params: {'regressor': AdaptiveRandomForestRegressor(n_models=20), 'regressor__n_models': 20}


#### Save model


In [8]:
auto_pipeline.save_model(".")

[2023-07-11 21:13:43:968] - INFO - SAIL (SAILAutoPipeline) : making directory tree ./sail_auto_pipeline
[2023-07-11 21:13:45:215] - INFO - SAIL (SAILPipeline) : making directory tree ./sail_auto_pipeline/pipeline_strategy/best_pipeline


'./sail_auto_pipeline'

#### Load model


In [9]:
new_auto_pipeline = SAILAutoPipeline.load_model(".")

[2023-07-11 21:13:46:516] - INFO - SAIL : Pipeline Strategy [DetectAndIncrement] created with actions: ['DATA_COLLECTION', 'FIND_BEST_PIPELINE', 'SCORE_AND_DETECT_DRIFT', 'PARTIAL_FIT_MODEL']


#### Continue trainig using the load model


In [10]:
for start in range(1000, 2001, batch_size):

    end = start + batch_size

    X_train = X.iloc[start:end]
    y_train = y.iloc[start:end]

    if end > 1000: # search_data_size is 1000
        preds = new_auto_pipeline.predict(X_train)
        if preds is not None:
            y_preds.extend(list(preds))
            y_true.extend(list(y_train))

    new_auto_pipeline.train(X_train, y_train)


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.165s, Batch Size=50, P_Score=0.646]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.147s, Batch Size=50, P_Score=0.789]            


>>>--------------------------------------------------------------------------------------------


...

SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.189s, Batch Size=50, P_Score=0.714]            

>>>--------------------------------------------------------------------------------------------



SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.198s, Batch Size=50, P_Score=0.724]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.194s, Batch Size=50, P_Score=0.702]            


#### Final Score


In [11]:
new_auto_pipeline.progressive_score

0.7024539491475816

### Plot predictions


In [12]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds})
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()